In [97]:
import glob
import os
import pandas as pd

In [98]:
caminho_raw = "../dados/raw/"

arquivos_bacen_resumo = "../dados/bacen/resumo_bacen/resumo_*.csv"
arquivos_bacen_dre = "../dados/bacen/dre_bacen/dre_*.csv"

lista_arquivo_resumo = glob.glob(arquivos_bacen_resumo)
lista_arquivo_dre = glob.glob(arquivos_bacen_dre)

In [99]:
lista_resumo = []

for arquivo_resumo in lista_arquivo_resumo:
    temp_df_resumo = pd.read_csv(arquivo_resumo, sep=";", encoding='utf-8', low_memory=False)
    lista_resumo.append(temp_df_resumo)

df_resumo_total = pd.concat(lista_resumo, ignore_index=True)

df_resumo_total.columns = df_resumo_total.columns.str.strip()

In [100]:
df_resumo = df_resumo_total.copy()

df_resumo['Carteira de Crédito'] = df_resumo['Carteira de Crédito'].fillna(df_resumo['Carteira de Crédito Classificada'])
df_resumo['Passivo Exigível'] = df_resumo['Passivo Exigível'].fillna(df_resumo['Passivo Circulante e Exigível a Longo Prazo e Resultados de Exercícios Futuros'])

colunas_excluir = ['Carteira de Crédito Classificada','Conglomerado Prudencial.1', 'Passivo Circulante e Exigível a Longo Prazo e Resultados de Exercícios Futuros', 'Títulos e Valores Mobiliários', 'Unnamed: 19', 'Unnamed: 20', 'Conglomerado Financeiro']

df_resumo = df_resumo.drop(columns=colunas_excluir)

df_resumo = df_resumo.dropna(subset='Cidade')

df_resumo.to_parquet(os.path.join(caminho_raw, "resumo.parquet"), index=False)

df_resumo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17170 entries, 0 to 17939
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Instituição                      17170 non-null  object 
 1   Código                           17170 non-null  float64
 2   Conglomerado Prudencial          8373 non-null   object 
 3   TCB                              17170 non-null  object 
 4   TC                               17170 non-null  float64
 5   TI                               17170 non-null  float64
 6   Cidade                           17170 non-null  object 
 7   UF                               17170 non-null  object 
 8   Data                             17170 non-null  object 
 9   Ativo Total                      17170 non-null  object 
 10  Captações                        17170 non-null  object 
 11  Patrimônio Líquido               17170 non-null  object 
 12  Lucro Líquido          

In [101]:
lista_dre = []

for arquivo_dre in lista_arquivo_dre:
    temp_df_dre = pd.read_csv(arquivo_dre, sep=";", encoding='utf-8', low_memory=False)
    lista_dre.append(temp_df_dre)

df_dre_total = pd.concat(lista_dre, ignore_index=True)

df_dre_total.columns = df_dre_total.columns.str.strip()

In [102]:
df_dre = df_dre_total.copy()

df_dre['Resultado Antes Impostos'] = df_dre['Resultado antes da Tributação e Participações (x) = (k) + (l) + (w)'].fillna(df_dre['Resultado antes da Tributação, Lucro e Participação (g) = (e) + (f)'])
df_dre['Lucro Líquido'] = df_dre['Lucro Líquido (aa) = (x) + (y) + (z)'].fillna(df_dre['Lucro Líquido (j) = (g) + (h) + (i)'])
df_dre['Outras Receitas / Despesas'] = df_dre['Outras Receitas / Despesas'].fillna(df_dre['Outras Receitas/Despesas Operacionais'])
df_dre['Participações no Lucro (z)'] = df_dre['Participações no Lucro (z)'].fillna(df_dre['Participação nos Lucros (i)'])
df_dre['Imposto de Renda e Contribuição Social (y)'] = df_dre['Imposto de Renda e Contribuição Social (y)'].fillna(df_dre['Imposto de Renda e Contribuição Social (h)'])
df_dre['Resultado com Transações de Pagamento'] = df_dre['Resultado com Transações de Pagamento'].fillna('0')

colunas_vazias = [c for c in df_dre.columns if c.startswith('Unnamed')]
colunas_excluir = ['Conglomerado Financeiro', 'Conglomerado Prudencial.1', 'Ativo Total', 'Resultado antes da Tributação e Participações (x) = (k) + (l) + (w)', 'Resultado antes da Tributação, Lucro e Participação (g) = (e) + (f)','Lucro Líquido (aa) = (x) + (y) + (z)','Lucro Líquido (j) = (g) + (h) + (i)','Outras Receitas/Despesas Operacionais','Imposto de Renda e Contribuição Social (h)','Participação nos Lucros (i)','Juros Sobre Capital Social de Cooperativas (k)','Resultado Operacional (e) = (c) + (d)','Resultado Não Operacional (f)']

colunas_exclusão = colunas_vazias + colunas_excluir

df_dre = df_dre.dropna(subset='Cidade')
df_dre = df_dre.drop(columns=colunas_exclusão)

df_dre.to_parquet(os.path.join(caminho_raw, "dre.parquet"), index=False)

df_dre.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17165 entries, 2 to 17956
Data columns (total 16 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Instituição                                 17165 non-null  object 
 1   Código                                      17165 non-null  float64
 2   Conglomerado Prudencial                     8361 non-null   object 
 3   TCB                                         17165 non-null  object 
 4   TC                                          17165 non-null  float64
 5   TI                                          17165 non-null  float64
 6   Cidade                                      17165 non-null  object 
 7   UF                                          17165 non-null  object 
 8   Data                                        17165 non-null  object 
 9   Resultado de Intermediação Financeira       17165 non-null  object 
 10  Resultado com T